https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [1]:
import glob
import pandas as pd
import numpy as np
import json

In [2]:
annotations=glob.glob('data/OPP-115/annotations/*')

In [3]:
annotations_list=[]
for file in annotations:
    annotations_list.append(pd.read_csv(file, header=None))

In [4]:
annot=pd.concat(annotations_list, ignore_index=True)

In [5]:
annot.columns = ['annotation_ID', 'batch_ID', 'annotator_ID', 'policy_ID', 'segment_ID','category_name',
            'attribute_value_pairs','date','policy_URL']

In [6]:
annot['dict'] = annot.attribute_value_pairs.apply(lambda x: json.loads(x))

In [7]:
annot.head()

,annotation_ID,batch_ID,annotator_ID,policy_ID,segment_ID,category_name,attribute_value_pairs,date,policy_URL,dict
0,13160,test_category_labeling_highlight_fordham_aaaa,121,3828,0,Other,"{""Other Type"": {""endIndexInSegment"": 575, ""sta...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Other Type': {'endIndexInSegment': 575, 'sta..."
1,13161,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
2,13162,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
3,13163,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."
4,13164,test_category_labeling_highlight_fordham_aaaa,121,3828,1,First Party Collection/Use,"{""Collection Mode"": {""endIndexInSegment"": -1, ...",5/7/15,http://www.kraftrecipes.com/about/privacynotic...,"{'Collection Mode': {'endIndexInSegment': -1, ..."


In [8]:
for x in annot['dict']:
    for key, value in x.items():
        value.pop('endIndexInSegment', None)
        value.pop('startIndexInSegment', None)

In [9]:
text=[]
category=[]
subcat=[]
label=[]
counter=0
for i,x in enumerate(annot['dict']):
    for key, value in x.items():
        subcat.append(key)
        if value.get('selectedText')==None:
            text.append('noSelectedText')
        else:
            text.append(value.get('selectedText'))  
        category.append(annot['category_name'][i])
        for k, v in value.items():
            if k=='value':
                label.append(v)

In [10]:
d=list(zip(text, category, subcat, label))

In [11]:
data=pd.DataFrame(d, columns=['text', 'category', 'subcategory', 'label'])

In [12]:
data.head()
data.iloc[3].text

'register on our website or participate in our offers and programs'

In [13]:
data['cat_sub'] = data['category'] +'-'+ data['subcategory']

In [14]:
data.groupby('cat_sub').count()

,text,category,subcategory,label
cat_sub,,,,
Data Retention-Personal Information Type,370,370,370,370
Data Retention-Retention Period,370,370,370,370
Data Retention-Retention Purpose,370,370,370,370
Data Security-Security Measure,1008,1008,1008,1008
Do Not Track-Do Not Track policy,90,90,90,90
First Party Collection/Use-Action First-Party,8935,8935,8935,8935
First Party Collection/Use-Choice Scope,8935,8935,8935,8935
First Party Collection/Use-Choice Type,8935,8935,8935,8935
First Party Collection/Use-Collection Mode,8935,8935,8935,8935


In [15]:
data.groupby(['category','subcategory']).nunique().count()

text           36
category       36
subcategory    36
label          36
cat_sub        36
dtype: int64

In [16]:
data['cat_sub_lab'] = data['category'] +'-'+ data['subcategory'] +'-'+ data['label']

In [17]:
import string, re
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct
data['text'] = data['text'].apply(lambda x : remove_punct(x.lower()))

data = data[data['text']!='null']
data = data[data['text']!='noselectedtext']
data = data[data['text']!='']
data = data[data['text']!=' ']
data.reset_index(inplace=True, drop=True)
ds = data.drop_duplicates()

In [18]:
ds.shape

(36717, 6)

In [19]:
ds.groupby('category').count()

,text,subcategory,label,cat_sub,cat_sub_lab
category,,,,,
Data Retention,676,676,676,676,676
Data Security,808,808,808,808,808
Do Not Track,67,67,67,67,67
First Party Collection/Use,15346,15346,15346,15346,15346
International and Specific Audiences,582,582,582,582,582
Other,2759,2759,2759,2759,2759
Policy Change,932,932,932,932,932
Third Party Sharing/Collection,10655,10655,10655,10655,10655
"User Access, Edit and Deletion",1094,1094,1094,1094,1094


In [20]:
ds.groupby(['category','subcategory']).count().shape

(36, 4)

In [21]:
ds.groupby(['category','subcategory','label']).count().shape

(259, 3)

In [22]:
ds['tokenized'] = ds['text'].apply(lambda x : re.split(' ', x))
# ds['tokenized_text']=res

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
from gensim.models import Word2Vec
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [24]:
# w2v = Word2Vec(ds['tokenized'])
# w2v.save('word2vec.model')

In [25]:
w2v = Word2Vec.load('word2vec.model')

2019-09-20 17:57:28,927 : INFO : loading Word2Vec object from word2vec.model
/anaconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-09-20 17:57:28,972 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2019-09-20 17:57:28,973 : INFO : setting ignored attribute vectors_norm to None
2019-09-20 17:57:28,975 : INFO : loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
2019-09-20 17:57:28,976 : INFO : loading trainables recursively from word2vec.model.trainables.* with mmap=None
2019-09-20 17:57:28,977 : INFO : setting ignored attribute cum_table to None
2019-09-20 17:57:28,979 : INFO : loaded word2vec.model


In [26]:
from sklearn.preprocessing import OneHotEncoder
# encode = OneHotEncoder()

In [27]:
ds.reset_index(inplace=True, drop=True)

In [28]:
word_vectors = w2v.wv

In [29]:
word2id = {k: v.index for k, v in word_vectors.vocab.items()}

In [30]:
def add_new_word(new_word, new_vector, new_index, embedding_matrix, word2id):
    embedding_matrix = np.insert(embedding_matrix, [new_index], [new_vector], axis=0)
    
    word2id = {word: (index+1) if index >= new_index else index for word, index in word2id.items()}
    word2id[new_word] = new_index
    return embedding_matrix, word2id

UNK_INDEX = 0
UNK_TOKEN = 'UNK'

embedding_matrix = word_vectors.vectors
unk_vector = embedding_matrix.mean(0)
embedding_matrix, word2id = add_new_word(UNK_TOKEN, unk_vector, UNK_INDEX, embedding_matrix, word2id)

In [31]:
len(word2id)

3933

In [32]:
embedding_matrix.shape

(3933, 100)

In [33]:
embedding_matrix[0].shape

(100,)

In [34]:
embedding_matrix = np.insert(embedding_matrix, len(embedding_matrix), [np.zeros((100,))], axis=0)

In [35]:
len(embedding_matrix)

3934

In [36]:
embedding_matrix[3933].shape

(100,)

In [37]:
def get_int_data(token_text, word2id):
    x = []
    unk_count = 0
    for item in token_text:
        temp=[]
        x.append(temp)
        for word in item:
            if word in word2id:
                temp.append(word2id.get(word))
            else:
                temp.append(UNK_INDEX)
                unk_count += 1
    print('Data created. Percentage of unknown words: %.3f' % (unk_count))
    return np.array(x)

x=get_int_data(ds.tokenized, word2id)
x.shape

Data created. Percentage of unknown words: 7006.000


(36717,)

In [38]:
ds['enumerated_text']=x

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
#ds['enumerated_text']=ds['enumerated_text'].apply(lambda x: np.array(x))

In [40]:
target = ds.category

In [41]:
ds['Do_Not_Track'] = ds['category'].apply(lambda x: 1 if x == 'Do Not Track' else 0)
ds['Other'] = ds['category'].apply(lambda x: 1 if x == 'Other' else 0)
ds['First_Party_Collection_Use'] = ds['category'].apply(lambda x: 1 if x == 'First Party Collection/Use' else 0)
ds['Third_Party_Sharing_Collection'] = ds['category'].apply(lambda x: 1 if x == 'Third Party Sharing/Collection' else 0)
ds['User_Choice_Control'] = ds['category'].apply(lambda x: 1 if x == 'User Choice/Control' else 0)
ds['International_and_Specific_Audiences'] = ds['category'].apply(lambda x: 1 if x == 'International and Specific Audiences' else 0)
ds['Data_Security'] = ds['category'].apply(lambda x: 1 if x == 'Data Security' else 0)
ds['Policy_Change'] = ds['category'].apply(lambda x: 1 if x == 'Policy Change' else 0)
ds['Data_Retention'] = ds['category'].apply(lambda x: 1 if x == 'Data Retention' else 0)
ds['User_Access_Edit_and_Deletion'] = ds['category'].apply(lambda x: 1 if x == 'User Access, Edit and Deletion' else 0)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [42]:
# sanity check
print(ds['Do_Not_Track'].sum())
print(ds['Other'].sum())
print(ds['First_Party_Collection_Use'].sum())
print(ds['Third_Party_Sharing_Collection'].sum())
print(ds['User_Choice_Control'].sum())
print(ds['International_and_Specific_Audiences'].sum())
print(ds['Data_Security'].sum())
print(ds['Policy_Change'].sum())
print(ds['Data_Retention'].sum())
print(ds['User_Access_Edit_and_Deletion'].sum())
ds.head()

67
2759
15346
10655
3798
582
808
932
676
1094


,text,category,subcategory,label,cat_sub,cat_sub_lab,tokenized,enumerated_text,Do_Not_Track,Other,First_Party_Collection_Use,Third_Party_Sharing_Collection,User_Choice_Control,International_and_Specific_Audiences,Data_Security,Policy_Change,Data_Retention,User_Access_Edit_and_Deletion
0,effective date may 7 2015 kraft site privacy n...,Other,Other Type,Introductory/Generic,Other-Other Type,Other-Other Type-Introductory/Generic,"[effective, date, may, 7, 2015, kraft, site, p...","[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2...",0,1,0,0,0,0,0,0,0,0
1,collec,First Party Collection/Use,Choice Scope,Collection,First Party Collection/Use-Choice Scope,First Party Collection/Use-Choice Scope-Collec...,[collec],[2151],0,0,1,0,0,0,0,0,0,0
2,register on our website or participate in our ...,First Party Collection/Use,Action First-Party,Collect on website,First Party Collection/Use-Action First-Party,First Party Collection/Use-Action First-Party-...,"[register, on, our, website, or, participate, ...","[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]",0,0,1,0,0,0,0,0,0,0
3,personallyidentifiable information such as,First Party Collection/Use,Personal Information Type,Generic personal information,First Party Collection/Use-Personal Informatio...,First Party Collection/Use-Personal Informatio...,"[personallyidentifiable, information, such, as]","[630, 5, 31, 23]",0,0,1,0,0,0,0,0,0,0
4,if you choose to register on our website or pa...,First Party Collection/Use,Choice Type,Opt-in,First Party Collection/Use-Choice Type,First Party Collection/Use-Choice Type-Opt-in,"[if, you, choose, to, register, on, our, websi...","[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1...",0,0,1,0,0,0,0,0,0,0


In [43]:
category_df = ds[['enumerated_text','Do_Not_Track','Other','First_Party_Collection_Use','Third_Party_Sharing_Collection',\
                  'User_Choice_Control','International_and_Specific_Audiences','Data_Security','Policy_Change','Data_Retention',\
                 'User_Access_Edit_and_Deletion']]
category_df.head()

,enumerated_text,Do_Not_Track,Other,First_Party_Collection_Use,Third_Party_Sharing_Collection,User_Choice_Control,International_and_Specific_Audiences,Data_Security,Policy_Change,Data_Retention,User_Access_Edit_and_Deletion
0,"[553, 244, 11, 2150, 839, 1332, 35, 26, 139, 2...",0,1,0,0,0,0,0,0,0,0
1,[2151],0,0,1,0,0,0,0,0,0,0
2,"[204, 19, 10, 52, 6, 212, 13, 10, 168, 3, 363]",0,0,1,0,0,0,0,0,0,0
3,"[630, 5, 31, 23]",0,0,1,0,0,0,0,0,0,0
4,"[32, 4, 105, 1, 204, 19, 10, 52, 6, 212, 13, 1...",0,0,1,0,0,0,0,0,0,0


In [44]:
from keras.preprocessing.sequence import pad_sequences
max_length = max(category_df.enumerated_text.apply(lambda x: len(x)))
padded_docs = pad_sequences(category_df.enumerated_text, maxlen=max_length, padding='post', value=(len(embedding_matrix)-1))

Using TensorFlow backend.


In [45]:
padded_docs[0].shape

(289,)

In [46]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, Activation, Flatten, Dropout
from keras.models import load_model

In [47]:
from keras import regularizers

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Do_Not_Track, test_size=0.3, random_state = 0)


# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 30.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.05)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=5, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('do_not_track.h5')
model = load_model('do_not_track.h5')

from sklearn.metrics import confusion_matrix, f1_score, recall_score
print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

2019-09-20 17:57:33,875 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
2019-09-20 17:57:34,346 : WARNING : From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


[[25645     8]
 [    0    48]]
0.923076923076923
1.0
[[10993     4]
 [    5    14]]
0.7567567567567567
0.7368421052631579


In [48]:
# Other Model
x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Other, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('other.h5')
model = load_model('other.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

2019-09-20 17:57:45,312 : WARNING : From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


[[22175  1566]
 [   39  1921]]
0.7053423903065908
0.9801020408163266
[[9298  919]
 [ 182  617]]
0.5284796573875803
0.772215269086358


In [49]:
#  First Party Collection/Use Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.First_Party_Collection_Use, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 10.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('first_party_collection_use.h5')
model = load_model('first_party_collection_use.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[ 7028  7949]
 [  107 10617]]
0.7249573233185387
0.9900223797090638
[[2771 3623]
 [ 157 4465]]
0.7025963808025176
0.9660320207702293


In [50]:
#  Data Retention Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Data_Retention, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 40.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dropout(0.2))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('data_retention.h5')
model = load_model('data_retention.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[21861  3356]
 [    7   477]]
0.22098679638637944
0.9855371900826446
[[9243 1581]
 [  58  134]]
0.140534871525957
0.6979166666666666


In [51]:
#  Data Security Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Data_Security, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 40.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('data_security.h5')
model = load_model('data_security.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[24307   843]
 [    4   547]]
0.5636269963936115
0.9927404718693285
[[10304   455]
 [   40   217]]
0.46716899892357383
0.8443579766536965


In [52]:
#  International and Specific Audiences Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.International_and_Specific_Audiences, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 45.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('internation_specific_audiences.h5')
model = load_model('internation_specific_audiences.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[24949   357]
 [    5   390]]
0.6830122591943957
0.9873417721518988
[[10638   191]
 [   39   148]]
0.5627376425855514
0.7914438502673797


In [53]:
#  Policy Change Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Policy_Change, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 40.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(50, activation='tanh', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('policy_change.h5')
model = load_model('policy_change.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[24904   149]
 [    0   648]]
0.896885813148789
1.0
[[10608   124]
 [   62   222]]
0.7047619047619047
0.7816901408450704


In [54]:
#  Third Party Sharing/Collection Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.Third_Party_Sharing_Collection, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 20}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('third_party_sharing_collection.h5')
model = load_model('third_party_sharing_collection.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[13936  4302]
 [    6  7457]]
0.7758818021017584
0.9991960337665818
[[5222 2602]
 [ 592 2600]]
0.619490111984751
0.8145363408521303


In [55]:
#  User Access, Edit and Deletion Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Access_Edit_and_Deletion, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 15.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_access_edit_deletion.h5')
model = load_model('user_access_edit_deletion.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[23907  1057]
 [   41   696]]
0.5590361445783133
0.9443690637720489
[[10118   541]
 [   87   270]]
0.4623287671232877
0.7563025210084033


In [56]:
#  User Choice/Control Model

x_train, x_test, y_train, y_test = train_test_split(padded_docs, category_df.User_Choice_Control, test_size=0.3, random_state = 0)

# vocab_length = len(embedding_matrix)

# class_weights = {0: 1.,
#                 1: 40.}

# model = Sequential()
# model.add(Embedding(input_dim=vocab_length,
#                    output_dim=100,
#                    weights=[embedding_matrix],
#                    input_length=padded_docs.shape[1]))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(50, activation='tanh'))
# model.add(Dense(1, activation='sigmoid'))
# #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss="binary_crossentropy",
#               optimizer='adam',
#               metrics=['accuracy'])

# history=model.fit(x_train, y_train, epochs=10, batch_size=100,\
#                   validation_data=(x_test, y_test),class_weight=class_weights)

# model.save('user_choice_control.h5')
model = load_model('user_choice_control.h5')

print(confusion_matrix(y_train,model.predict_classes(x_train)))
print(f1_score(y_train, model.predict_classes(x_train)))
print(recall_score(y_train, model.predict_classes(x_train)))
print(confusion_matrix(y_test,model.predict_classes(x_test)))
print(f1_score(y_test, model.predict_classes(x_test)))
print(recall_score(y_test, model.predict_classes(x_test)))

[[19734  3276]
 [    2  2689]]
0.6213031423290203
0.9992567818654775
[[8154 1755]
 [ 305  802]]
0.43777292576419213
0.7244805781391147
